# Outages
**Name(s)**: Bill Wang, Ethan Cao

**Website Link**: https://billwang04.github.io/us_state_power_outage/

## Code

In [8]:
import pandas as pd
import numpy as np
import os


import plotly.express as px
pd.options.plotting.backend = 'plotly'

First we load in the data from the excel file and combine the time and date columns

In [9]:
def combine_times(date_col_name, time_col_name, new_col_name, df):
    df = df.copy()
    df[new_col_name] = df[date_col_name] + pd.to_timedelta(df[time_col_name].astype(str))
    return df

In [10]:
data = pd.read_excel("outage.xlsx", skiprows=[0,1,2,3,4,6], index_col=1).iloc[:,1:]
data = combine_times("OUTAGE.START.DATE", 'OUTAGE.START.TIME', 'OUTAGE.START.DATETIME', data)
data = combine_times("OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME", "OUTAGE.RESTORATION.DATETIME", data)

In [11]:
data.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

In [12]:
# data = data[['U.S._STATE',"YEAR",'CLIMATE.REGION', 'OUTAGE.START.DATETIME', 'OUTAGE.RESTORATION.DATETIME','CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION','DEMAND.LOSS.MW']]

In [13]:
univariant_plot = px.histogram(data['OUTAGE.DURATION'])
univariant_plot.update_layout(xaxis_title = 'OUTAGE.DURATION in Minutes', showlegend = False, title = 'Count of Duration of Outage')

In [14]:
univariant_plot.write_html('../uni-plot.html', include_plotlyjs='cdn')

In [15]:
bivariant = data.plot(kind = 'bar', x = 'U.S._STATE', y = 'OUTAGE.DURATION')

In [16]:
bivariant = data.groupby('U.S._STATE')['OUTAGE.DURATION'].sum().sort_values().reset_index().plot(kind = 'bar', x ='U.S._STATE' , y= 'OUTAGE.DURATION')

In [17]:
bivariant.update_xaxes(dtick=1)

In [18]:
bivariant.write_html('../bi-plot.html', include_plotlyjs='cdn')

In [19]:
data.groupby('U.S._STATE')['YEAR'].count().sort_values()

U.S._STATE
Alaska                    1
South Dakota              2
North Dakota              2
Montana                   3
Mississippi               4
West Virginia             4
Nebraska                  4
Hawaii                    5
Alabama                   6
Wyoming                   6
Nevada                    7
South Carolina            8
New Mexico                8
Iowa                      8
Vermont                   9
Idaho                     9
Kansas                    9
District of Columbia     10
Kentucky                 13
New Hampshire            14
Minnesota                15
Colorado                 15
Missouri                 17
Georgia                  17
Massachusetts            18
Connecticut              18
Maine                    19
Wisconsin                20
Oklahoma                 24
Arkansas                 25
Oregon                   26
Arizona                  28
Tennessee                34
New Jersey               35
Virginia                 37
Louisiana

In [20]:
data[['CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL']][data['CAUSE.CATEGORY.DETAIL'].isna()]

,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL
OBS,,
1,severe weather,NaN
5,severe weather,NaN
19,severe weather,NaN
20,severe weather,NaN
24,intentional attack,NaN
...,...,...
1523,system operability disruption,NaN
1525,public appeal,NaN
1530,public appeal,NaN


### Question
1. Out of Nerc Regions/ **States** , what are the most likely to have the worst outages, what are the causes.
2. UniVariant: 
3. BiVariant: 

In [21]:
# data[data[''].isnull()]

### Deadlines
1. Sat: Question, Cleaning & EDA, Plan for Univariant and Bivariant
2. Sun: Assignment of Missigness: Have Univariant and Bivariant done
3. Mon: Hypothesis Testing
4. Tue: Start the report 

In [22]:
data['CAUSE.CATEGORY'][data['CAUSE.CATEGORY'].isna()]

Series([], Name: CAUSE.CATEGORY, dtype: object)

### Cleaning and EDA

### Assessment of Missingness

## NMAR

**CAUSE CATEGORY**:
All the NAN values in cause category could nan because they could pinpoint the reason for the outage, therefore we can say that the missingness in the Cause Category must be dependent on itself. 

States and CUstomers.affected

## MAR

### US STATE WITH CUSTOMERS
**NULL HYPOTHESIS**: There is no significant difference between 

In [28]:
distr_nan = data[['U.S._STATE', 'CUSTOMERS.AFFECTED']].assign(NotNA = data[['U.S._STATE', 'CUSTOMERS.AFFECTED']]['CUSTOMERS.AFFECTED'].isna()== False)
dist_notNA = distr_nan[distr_nan['NotNA']][['U.S._STATE','NotNA']]
dist_notNA = dist_notNA.groupby('U.S._STATE').count()
dist_NA = distr_nan[distr_nan['NotNA'] == False][['U.S._STATE','NotNA']].rename(columns = {'NotNA' : 'ISNA'})
dist_NA = dist_NA.groupby('U.S._STATE').count()
plot_df = dist_NA.merge(dist_notNA, left_index=True, right_index=True)
plot_dist_na =  px.bar(plot_df.reset_index(), x='U.S._STATE', y=['ISNA', 'NotNA'], title='Distribution of NAN and Non_NAN values for each state')
plot_dist_na.update_layout(barmode='group', xaxis_tickangle=-45)
plot_dist_na

In [23]:
def find_tvd(df, depends_on, col_analyze):
    df = df.copy()
    df_needed = df.loc[:,[depends_on, col_analyze]]
    df_needed['ISNA'] = df_needed.loc[:,col_analyze].isna()
    df_needed[col_analyze] = df_needed.loc[:,col_analyze].fillna(0)
    find_prop = df_needed.pivot_table(index = depends_on, columns = 'ISNA', aggfunc = 'count', fill_value=0).loc[:,col_analyze]
    total_not_missing = find_prop[False].sum()
    total_missing = find_prop[True].sum()
    find_prop[False] = find_prop[False] / total_not_missing
    find_prop[True] = find_prop[True] / total_missing
    return find_prop.diff(axis=1)[True].abs().sum()

In [24]:
def mar_permutation(df, depends_on, col_analyze, N = 1000):
    observed = find_tvd(df, depends_on, col_analyze)
    arr = []
    for _ in range(N):
        shuffled = df.assign(**{col_analyze: np.random.permutation(df[col_analyze])}) 
        arr.append(find_tvd(shuffled, depends_on, col_analyze))
    plot = px.histogram(np.array(arr))
    plot.add_vline(x=observed, line_color= "green", annotation_text="obs")
    plot.update_layout(xaxis_title = "TVD of Missing and Non-Missing Values", yaxis_title = "Frequency", title_text = "Simulated Null Values",showlegend=False)


    return plot, np.array([np.array(arr) > observed]).mean()


In [25]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaN,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


In [26]:
graph, p_value = mar_permutation(data, 'CLIMATE.CATEGORY', 'CUSTOMERS.AFFECTED')
# graph.write_html('../03-topic/mar-hist.html', include_plotlyjs='cdn')
print(p_value)
graph

0.366


In [27]:
graph, p_value = mar_permutation(data, 'U.S._STATE', 'CUSTOMERS.AFFECTED')
print(p_value)
graph

0.0


### Hypothesis Testing

1. Out of Nerc Regions/ **States** , what are the most likely to have the worst outages, what are the causes.

In [29]:
data[['U.S._STATE', 'CUSTOMERS.AFFECTED']].groupby('U.S._STATE').mean()

,CUSTOMERS.AFFECTED
U.S._STATE,
Alabama,94328.800000
Alaska,14273.000000
Arizona,64402.666667
Arkansas,47673.846154
California,201365.716535
Colorado,41060.636364
Connecticut,60339.230769
Delaware,3475.000000
District of Columbia,194709.222222


In [30]:
def calculate_outage_severity(data, state, var):

    output = data.groupby(state).mean()
    
    return output.loc[True,var]

In [31]:
def perm_test(data, state, var, n=1000):
    data = data.copy()[["U.S._STATE", var]]
    data[state] = data["U.S._STATE"] == state
    obs = calculate_outage_severity(data, state, var)
    test_stats = []
    for _ in range(n):
        value = np.random.permutation(data[var])
        shuffled = data.assign(**{var : value })
        trial = calculate_outage_severity(shuffled, state, var)
        test_stats.append(trial)
    
    return  (np.array(test_stats) >= obs).mean()

**This** represents texas has a difference. 

In [32]:
stat = perm_test(data, "Texas", 'OUTAGE.DURATION')
stat

0.409

In [33]:
all_p_values_duration = {}
all_p_values_customers = {}
for i in data['U.S._STATE'].unique():
    all_p_values_duration[i] = perm_test(data, i, "OUTAGE.DURATION")
    all_p_values_customers[i] = perm_test(data, i , "CUSTOMERS.AFFECTED")

create dataframe to look p_value and how many outages to find how likely an outage is gonna happen and how severe it is. 

In [34]:
initial = pd.DataFrame({'duration_p_value' : pd.Series(all_p_values_duration),
              'customer_p_value' : pd.Series(all_p_values_customers)})
resulting_df = data.groupby('U.S._STATE')[['YEAR']].count().merge(initial, left_index = True, right_index = True).rename(columns = {'YEAR': 'Count of Outages'})
graph_df = resulting_df[(resulting_df['duration_p_value'] < 0.05) | (resulting_df['customer_p_value'] < 0.05 )].sort_values('customer_p_value')

In [35]:
graph_df

,Count of Outages,duration_p_value,customer_p_value
Montana,3,0.980,0.000
South Dakota,2,0.853,0.000
Texas,127,0.409,0.003
California,210,1.000,0.006
Florida,45,0.065,0.012
New York,71,0.001,0.119
Michigan,95,0.000,0.359
Alaska,1,0.000,0.529
Wisconsin,20,0.010,0.992


In [36]:
fig = px.bar(graph_df.reset_index().rename(columns = {'index': 'U.S._STATES'}), x='U.S._STATES', y=['duration_p_value', 'customer_p_value'], title='Grouped Bar Chart of Duration P-Value and Customer P-Value by U.S. States with a P-Value less than 0.05')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

In [37]:
fig.write_html('../p_value_bar.html', include_plotlyjs='cdn')

In [38]:
compare_df = pd.DataFrame(all_p_values_duration, index = np.arange(0,50)).melt().groupby('variable').max().sort_values('value')

In [39]:
compare_df#.merge(pd.DataFrame(all_p_values_customers, index = np.arange(0,50)).melt().groupby('variable').max().sort_values('value'))

,value
variable,
Alaska,0.000
Michigan,0.000
New York,0.001
Wisconsin,0.010
New Jersey,0.050
West Virginia,0.059
Arizona,0.062
Florida,0.065
Louisiana,0.071


In [ ]:
compare_df = compare_df.merge(data.groupby('U.S._STATE')['YEAR'].count().sort_values(ascending= False),left_index = True, right_index = True)

In [ ]:
customer_df =pd.DataFrame(all_p_values_customers, index = np.arange(0,50)).T[[0]].rename(columns = {0: 'customer_p_value'})

In [ ]:
customer_df

In [ ]:
customer_duration_df = compare_df.merge(customer_df, left_index= True, right_index= True)

In [ ]:
avg_dur = data['OUTAGE.DURATION'].mean()
avg_cus = data['CUSTOMERS.AFFECTED'].mean()


dir_duration = data.groupby("U.S._STATE")['OUTAGE.DURATION'].agg(lambda ser: ser.mean() - avg_dur)
dir_customer = data.groupby("U.S._STATE")['CUSTOMERS.AFFECTED'].agg(lambda ser: ser.mean() - avg_cus)

In [ ]:
amount_outages = customer_duration_df.rename(columns = {"YEAR": 'amount_outage'}).sort_values(by="value", ascending=True)
item = amount_outages#[amount_outages["amount_outage"] > 10]

item["dir_duration"] = dir_duration
item['dir_customer'] = dir_customer
item = item.reset_index().rename(columns = {"index" : 'U.S._STATE'}).fillna(0).sort_values(by = 'customer_p_value')

In [ ]:
resulting_df = item.set_index('U.S._STATE')[['value','customer_p_value']].rename(columns= {'value':'duration_p_value'}).sort_index()

In [ ]:
graph_df = resulting_df[(resulting_df['duration_p_value'] < 0.05) | (resulting_df['customer_p_value'] < 0.05 )].sort_values('customer_p_value')

In [ ]:
graph_df

In [ ]:
fig = px.bar(graph_df.reset_index(), x='U.S._STATE', y=['duration_p_value', 'customer_p_value'], title='Grouped Bar Chart of Duration P-Value and Customer P-Value by U.S. States with a P-Value less than 0.05')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

In [ ]:
# fig = px.bar(item, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
# fig.update_layout(barmode='group', xaxis_tickangle=-45)
# fig.show()

In [ ]:
demand_loss_mean = data.groupby('U.S._STATE')['DEMAND.LOSS.MW'].mean().fillna(0)
item = item.merge(demand_loss_mean, left_on= 'U.S._STATE', right_index = True)

In [ ]:
fig = px.bar(item, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [ ]:
item

In [ ]:
# # Example weights for components
# weight_outage_duration = 0.4
# weight_amount_duration = 0.3
# weight_customer_pvalue = 0.3




# # Calculate the weighted mean composite score
# item['Composite_Score'] =  (item['DEMAND.LOSS.MW'] * amount_outages)
# item

In [ ]:
item#.sort_values(by = 'Composite_Score').plot(kind = 'bar', x= 'U.S._STATE', y = "Composite_Score")

In [ ]:
df_both = item[(item['value'] < 0.05) | (item['customer_p_value'] < 0.05 )]
df_both

In [ ]:
fig = px.bar(df_both, x='U.S._STATE', y=['value', 'customer_p_value'], title='Grouped Bar Chart of Value and Amount_Outage by U.S. States')
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig

In [ ]:
fig.write_html('../p_value_bar.html', include_plotlyjs='cdn')